In [16]:
import boto3
import polars as pl

pl.Config.set_verbose(True)

import process_scop_sourmash_multisearch_polars

# Handwritten local modules
from polars_utils import scan_csv_sink_parquet, sink_parquet

In [17]:
s3 = boto3.resource("s3")

In [18]:
# !pip install boto3

In [3]:
csv = "s3://seanome-kmerseek/scope-benchmark/pipeline-outputs/2024-10-09__hp_k20-60/sourmash/multisearch/astral-scopedom-seqres-gd-sel-gs-bib-40-2.08.part_001.fa--in--astral-scopedom-seqres-gd-sel-gs-bib-40-2.08.part_001.fa.hp.10.multisearch.csv"

In [10]:
pq = scan_csv_sink_parquet(csv, verbose=True)
pq

/home/ec2-user/tmp/lecdzqve: 100%|█████████████████████████████████████████████████████████████████████████████████████| 94.4G/94.4G [24:20<00:00, 64.6MB/s]
/home/ec2-user/2024-kmerseek-analysis/notebooks/polars_utils.py:40: PerformanceWarning: Determining the column names of a LazyFrame requires resolving its schema, which is a potentially expensive operation. Use `LazyFrame.collect_schema().names()` to get the column names without this warning.
  f"\nWriting {df.select(pl.len()).collect().item()} rows and {len(df.columns)} columns to {pq} ..."



Writing 225197519 rows and 13 columns to s3://seanome-kmerseek/scope-benchmark/pipeline-outputs/2024-10-09__hp_k20-60/sourmash/multisearch/astral-scopedom-seqres-gd-sel-gs-bib-40-2.08.part_001.fa--in--astral-scopedom-seqres-gd-sel-gs-bib-40-2.08.part_001.fa.hp.10.multisearch.pq ...


RUN STREAMING PIPELINE
[csv -> parquet_sink]
STREAMING CHUNK SIZE: 3846 rows
/tmp/tmpnxp98qit: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 11.9G/11.9G [00:45<00:00, 262MB/s]


	Done.


's3://seanome-kmerseek/scope-benchmark/pipeline-outputs/2024-10-09__hp_k20-60/sourmash/multisearch/astral-scopedom-seqres-gd-sel-gs-bib-40-2.08.part_001.fa--in--astral-scopedom-seqres-gd-sel-gs-bib-40-2.08.part_001.fa.hp.10.multisearch.pq'

### Look at the difference in file sizes between the CSV and parquet files

In [14]:
! aws s3 ls --human-readable $csv

2024-10-11 23:54:22   87.9 GiB astral-scopedom-seqres-gd-sel-gs-bib-40-2.08.part_001.fa--in--astral-scopedom-seqres-gd-sel-gs-bib-40-2.08.part_001.fa.hp.10.multisearch.csv


In [15]:
! aws s3 ls --human-readable $pq

2024-10-29 21:16:54   11.1 GiB astral-scopedom-seqres-gd-sel-gs-bib-40-2.08.part_001.fa--in--astral-scopedom-seqres-gd-sel-gs-bib-40-2.08.part_001.fa.hp.10.multisearch.pq


# Apply to all hp files

## Read metadata

### Read Query Metadata

In [21]:
query_metadata = pl.scan_parquet(
    "s3://seanome-kmerseek/scope-benchmark/reference_files/scop.e.2.08.query_metadata.pq"
)
query_metadata

Async thread count: 4
/home/ec2-user/miniconda3/envs/2024-kmerseek-analysis-polars/lib/python3.12/site-packages/IPython/core/formatters.py:347: UserWarning: '(default_)region' not set; polars will try to get it from bucket

Set the region manually to silence this warning.
  return method()


### Read match metadata

In [22]:
match_metadata = pl.scan_parquet(
    "s3://seanome-kmerseek/scope-benchmark/reference_files/scop.e.2.08.match_metadata.pq"
)
match_metadata

## Iterate over all files

In [ ]:
# Skip ksizes 5 and 6 for now because those files are enormous, 2.3 GiB for k=5 and 175 MiB for k=6
# # -> Figure out how to use polars later
# ksizes = range(7, 21)
# moltype = "protein"


moltype_info = {
    # "protein": dict(
    #     ksizes=range(5, 21),
    #     pipeline_outdir="s3://seanome-kmerseek/scope-benchmark/pipeline-outputs/2024-10-08__protein_k5-20",
    #     analysis_outdir="s3://seanome-kmerseek/scope-benchmark/analysis-outputs/protein",
    # ),
    # "dayhoff": dict(
    #     ksizes=range(5, 21),
    #     pipeline_outdir="s3://seanome-kmerseek/scope-benchmark/pipeline-outputs/2024-10-09__dayhoff_k5-20",
    #     analysis_outdir="s3://seanome-kmerseek/scope-benchmark/analysis-outputs/dayhoff",
    # ),
    "hp": dict(
        ksizes=range(11, 20),
        pipeline_outdir="s3://seanome-kmerseek/scope-benchmark/pipeline-outputs/2024-10-09__hp_k20-60",
        analysis_outdir="s3://seanome-kmerseek/scope-benchmark/analysis-outputs/hp/",
    ),
}

for moltype, info in moltype_info.items():
    print(f"-- {moltype} --")
    ksizes = info["ksizes"]
    analysis_outdir = info["analysis_outdir"]
    pipeline_outdir = info["pipeline_outdir"]
    for ksize in ksizes:
        print(f"\t-- {ksize} --")
        try:
            parser = process_scop_sourmash_multisearch_polars.MultisearchParser(
                query_metadata=query_metadata,
                match_metadata=match_metadata,
                pipeline_outdir=pipeline_outdir,
                moltype=moltype,
                ksize=ksize,
                analysis_outdir=analysis_outdir,
                verbose=True,
            )
            csv = parser._make_multisearch_csv()
            scan_csv_sink_parquet(csv, verbose=True)
            # pq = csv.replace('.csv', '.pq')
            # if not
            # lf = parser.process_multisearch_scop_results()
            # lf.head().collect()
        except FileNotFoundError:
            pass
    # break
multisearch_metadata_filtered.head()

/home/ec2-user/tmp/i8i06qp3: 100%|██████████████████████████████████████████████████████████████████████████████████████| 89.7G/89.7G [10:34<00:00, 142MB/s]
/home/ec2-user/2024-kmerseek-analysis/notebooks/polars_utils.py:40: PerformanceWarning: Determining the column names of a LazyFrame requires resolving its schema, which is a potentially expensive operation. Use `LazyFrame.collect_schema().names()` to get the column names without this warning.
  f"\nWriting {df.select(pl.len()).collect().item()} rows and {len(df.columns)} columns to {pq} ..."
RUN STREAMING PIPELINE
[csv -> parquet_sink]
STREAMING CHUNK SIZE: 3846 rows



Writing 214441887 rows and 13 columns to s3://seanome-kmerseek/scope-benchmark/pipeline-outputs/2024-10-09__hp_k20-60/sourmash/multisearch/astral-scopedom-seqres-gd-sel-gs-bib-40-2.08.part_001.fa--in--astral-scopedom-seqres-gd-sel-gs-bib-40-2.08.part_001.fa.hp.11.multisearch.pq ...


/tmp/tmpwbe7r2yl: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 11.1G/11.1G [00:37<00:00, 298MB/s]


	Done.


/home/ec2-user/tmp/5q0woao9: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 80.3G/80.3G [09:04<00:00, 147MB/s]



Writing 191843183 rows and 13 columns to s3://seanome-kmerseek/scope-benchmark/pipeline-outputs/2024-10-09__hp_k20-60/sourmash/multisearch/astral-scopedom-seqres-gd-sel-gs-bib-40-2.08.part_001.fa--in--astral-scopedom-seqres-gd-sel-gs-bib-40-2.08.part_001.fa.hp.12.multisearch.pq ...


RUN STREAMING PIPELINE
[csv -> parquet_sink]
STREAMING CHUNK SIZE: 3846 rows
/tmp/tmpqajbcrc0: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9.65G/9.65G [00:35<00:00, 268MB/s]


	Done.


/home/ec2-user/tmp/4q9kd29g: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 65.5G/65.5G [11:30<00:00, 94.8MB/s]


In [ ]:
1+1